In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using FITSIO
using LinearAlgebra
using PythonCall
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaime/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

# T-Matrices

In [2]:
#println("My id is ", myid(), " and I have ", Threads.nthreads(), " threads")

fol = "DESY1"
data_set = "wlwl_Nzs"
meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);

nz_path = "../data/DESY1/Nzs/"
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESwl__0_e")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESwl__1_e")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESwl__2_e")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESwl__3_e")

(Float32[0.06, 0.099999994, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.42000002  …  1.26, 1.3, 1.34, 1.38, 1.4200001, 1.46, 1.5, 1.54, 1.5799999, 1.62], [4.629577847407024, 29.132412608452444, 21.765409190156518, 20.62146040771289, 49.575919814294636, 19.756458131549834, 28.671379304807797, 16.733425194984918, 36.12801036345403, 15.65029828338953  …  159.1343087971742, 45.68244851651424, 57.13816009303667, 74.52981085428384, 67.0412204252083, 69.0824797219035, 38.605059715930935, 18.387277685421846, 18.004397396845302, 19.840074411368867], [34.72926020369569 -0.3508402184688245 … -0.0035034826825542913 0.025896520365390033; -0.3508402184688245 527.8777602661406 … -0.09997382746286836 -0.09250459543441937; … ; -0.0035034826825542913 -0.09997382746286836 … 94.73249576839464 0.09124693778804871; 0.025896520365390033 -0.09250459543441937 … 0.09124693778804871 106.16024698844336])

In [3]:
function model_cov0(nz_k0;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov1(nz_k1;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov2(nz_k2;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov3(nz_k3;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

In [8]:
d = 0.0005
m  = length(nz_k0)
T0_num = zeros(240, m)
T1_num = zeros(240, m)
T2_num = zeros(240, m)
T3_num = zeros(240, m)

for i in 1:m
    e = zeros(m)
    e[i] = d
    T0_num[:, i] = (model_cov0(nz_k0+e) .- model_cov0(nz_k0)) ./ d
    T1_num[:, i] = (model_cov1(nz_k1+e) .- model_cov1(nz_k1)) ./ d
    T2_num[:, i] = (model_cov2(nz_k2+e) .- model_cov2(nz_k2)) ./ d
    T3_num[:, i] = (model_cov3(nz_k3+e) .- model_cov3(nz_k3)) ./ d
end

In [9]:
using ForwardDiff

In [10]:
T0 = ForwardDiff.jacobian(model_cov0, nz_k0);

In [11]:
T1 = ForwardDiff.jacobian(model_cov1, nz_k1);

In [12]:
T2 = ForwardDiff.jacobian(model_cov2, nz_k2);

In [13]:
T3 = ForwardDiff.jacobian(model_cov3, nz_k3);

In [14]:
(T0.-T0_num) ./ T0_num

240×40 Matrix{Float64}:
   6.42392e-8    1.1654e-7     1.16502e-7  …   -2.4885e-7    -1.26346e-7
   5.29337e-8    1.07602e-7    1.11495e-7      -3.34512e-7   -1.6867e-7
   7.48365e-8    1.22981e-7    1.19983e-7      -4.13471e-7   -2.19309e-7
   5.92982e-8    1.16507e-7    1.19524e-7      -4.68466e-7   -2.40713e-7
   5.80384e-8    1.14871e-7    1.14127e-7      -5.12388e-7   -2.64956e-7
   4.67663e-8    1.05924e-7    1.10863e-7  …   -5.48032e-7   -2.81462e-7
   6.07951e-8    1.15953e-7    1.13501e-7      -5.78323e-7   -3.02097e-7
   5.65562e-8    1.12041e-7    1.14576e-7      -6.05134e-7   -3.10299e-7
   5.31144e-8    1.15231e-7    1.216e-7        -6.28487e-7   -3.21213e-7
   6.15344e-8    1.19953e-7    1.19974e-7      -6.56427e-7   -3.39243e-7
   6.63751e-8    1.16379e-7    1.12862e-7  …   -6.79252e-7   -3.54508e-7
   5.93376e-8    1.16329e-7    1.16158e-7      -6.97903e-7   -3.62165e-7
   7.12854e-8    1.17786e-7    1.189e-7        -7.22368e-7   -3.74604e-7
   ⋮                        

In [22]:
TT0 = T0 * cov_k0 * T0'
TT1 = T1 * cov_k1 * T1'
TT2 = T2 * cov_k2 * T2'
TT3 = T3 * cov_k3 * T3';

TT0_num = T0_num * cov_k0 * T0_num'
TT1_num = T1_num * cov_k1 * T1_num'
TT2_num = T2_num * cov_k2 * T2_num'
TT3_num = T3_num * cov_k3 * T3_num';

In [35]:
isposdef(Symmetric(TT0_num))

false

In [38]:
eigen(Hermitian(Symmetric(TT0_num))).values

240-element Vector{Float64}:
 -1.1879500350496303e-37
 -8.250962301536381e-38
 -6.254603593167738e-38
 -2.3463529272951724e-38
 -1.653831728238613e-38
 -1.410744868169331e-38
 -1.079779403720184e-38
 -6.944359817295728e-39
 -3.300338152306984e-39
 -1.9169182373109744e-39
 -1.8376080756948184e-39
 -1.7152073195019638e-39
 -3.092048102436955e-40
  ⋮
  1.358781842796728e-29
  2.6366848603762536e-29
  5.231908928674761e-29
  8.383871475979268e-29
  1.4180213934409375e-28
  8.22728129212671e-28
  2.7162431820408832e-27
  9.982178745883075e-26
  2.646033660822745e-24
  3.4229377643242664e-23
  3.699194254215262e-21
  3.6167116891755605e-19

In [24]:
ishermitian(TT0)

false

In [25]:
issymmetric(TT0)

false

In [26]:
isposdef(Hermitian(TT0_num))

false

In [20]:
(Hermitian(TT0_num) - TT0_num) ./ TT0_num

240×240 Matrix{Float64}:
   0.0            0.0            0.0  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
  -1.60509e-16    0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
  -1.64694e-16    0.0            0.0  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
  -1.89532e-16    0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            1.70583e-16    0.0  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN
   0.0            0.0            0.0     NaN  NaN  NaN  NaN  NaN  NaN  NaN


In [ ]:
chain = sample(model(fake_data), NUTS(adaptation, TAP, ),
               MCMCDistributed(), iterations, nchains, progress=true; save_state=true)